In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
tf.random.set_seed(3)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:

data_set = pd.read_csv('UNR-IDD.csv')


In [3]:
data_set = data_set.sample(frac=1).reset_index(drop=True)

In [4]:
print(data_set)

                 Switch ID Port Number  Received Packets  Received Bytes  \
0      of:0000000000000007     Port#:3               579         6350091   
1      of:0000000000000001     Port#:2               480        12616018   
2      of:0000000000000001     Port#:3              4151        57018400   
3      of:0000000000000008     Port#:1                89           11979   
4      of:0000000000000001     Port#:3                93           13117   
...                    ...         ...               ...             ...   
37406  of:0000000000000002     Port#:1                13            1086   
37407  of:000000000000000c     Port#:1              3550        69443446   
37408  of:0000000000000007     Port#:2              1046        17005995   
37409  of:0000000000000008     Port#:1               486           62404   
37410  of:0000000000000005     Port#:1             46222         2504813   

       Sent Bytes  Sent Packets  Port alive Duration (S)  Packets Rx Dropped  \
0      

In [5]:
data_set['Actual Label'] = data_set['Label']+' '+data_set['Binary Label']

In [6]:
data_set.loc[data_set['Actual Label'] == 'TCP-SYN Attack','Actual Label']=0
data_set.loc[data_set['Actual Label'] == 'Blackhole Attack','Actual Label']=1
data_set.loc[data_set['Actual Label'] == 'Diversion Attack','Actual Label']=2
data_set.loc[data_set['Actual Label'] == 'Overflow Attack','Actual Label']=3
data_set.loc[data_set['Actual Label'] == 'Normal Normal','Actual Label']=4
data_set.loc[data_set['Actual Label'] == 'PortScan Attack','Actual Label']=5

In [7]:
data_set.head()

,Switch ID,Port Number,Received Packets,Received Bytes,Sent Bytes,Sent Packets,Port alive Duration (S),Packets Rx Dropped,Packets Tx Dropped,Packets Rx Errors,...,Latest bytes counter,is_valid,Table ID,Active Flow Entries,Packets Looked Up,Packets Matched,Max Size,Label,Binary Label,Actual Label
0,of:0000000000000007,Port#:3,579,6350091,25264075,1157,386,0,0,0,...,0,True,0,5,2458,2344,-1,Blackhole,Attack,1
1,of:0000000000000001,Port#:2,480,12616018,6342453,578,206,0,0,0,...,0,True,0,8,191605,191499,-1,PortScan,Attack,5
2,of:0000000000000001,Port#:3,4151,57018400,50695578,3693,2497,0,0,0,...,0,True,0,6,17621,17397,-1,Diversion,Attack,2
3,of:0000000000000008,Port#:1,89,11979,6315683,267,86,0,0,0,...,0,True,0,5,823,718,-1,PortScan,Attack,5
4,of:0000000000000001,Port#:3,93,13117,13243,96,101,0,0,0,...,0,True,0,6,577,485,-1,TCP-SYN,Attack,0


In [8]:
#Netoyage des donneés

In [ ]:
X= data_set.drop(['Switch ID', 'Port Number','is_valid','Label','Binary Label'], axis=1) 
Y = data_set['Actual Label']


In [10]:
var_th = VarianceThreshold(threshold = 0)
var_th.fit(X)

VarianceThreshold(threshold=0)

In [11]:
var_th.get_support()

array([ True,  True,  True,  True,  True, False, False, False, False,
        True,  True,  True,  True,  True, False, False, False, False,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True, False,  True])

In [12]:
X.columns[var_th.get_support()]

Index(['Received Packets', 'Received Bytes', 'Sent Bytes', 'Sent Packets',
       'Port alive Duration (S)', 'Delta Received Packets',
       'Delta Received Bytes', 'Delta Sent Bytes', 'Delta Sent Packets',
       'Delta Port alive Duration (S)', 'Connection Point', 'Total Load/Rate',
       'Total Load/Latest', 'Unknown Load/Rate', 'Unknown Load/Latest',
       'Latest bytes counter', 'Active Flow Entries', 'Packets Looked Up',
       'Packets Matched', 'Actual Label'],
      dtype='object')

In [13]:
const_columns = [column for column in X.columns
                if column not in X.columns[var_th.get_support()]
                ]
print(len(const_columns))

10


In [14]:
for features in const_columns:
    print(features)

Packets Rx Dropped
Packets Tx Dropped
Packets Rx Errors
Packets Tx Errors
Delta Packets Rx Dropped
 Delta Packets Tx Dropped
Delta Packets Rx Errors
Delta Packets Tx Errors
Table ID
Max Size


In [15]:
X.drop(const_columns, axis = 1)

,Received Packets,Received Bytes,Sent Bytes,Sent Packets,Port alive Duration (S),Delta Received Packets,Delta Received Bytes,Delta Sent Bytes,Delta Sent Packets,Delta Port alive Duration (S),Connection Point,Total Load/Rate,Total Load/Latest,Unknown Load/Rate,Unknown Load/Latest,Latest bytes counter,Active Flow Entries,Packets Looked Up,Packets Matched,Actual Label
0,579,6350091,25264075,1157,386,4,556,556,4,5,3,0,6889,0,6889,0,5,2458,2344,1
1,480,12616018,6342453,578,206,149,6039418,6771,94,5,2,0,0,0,0,0,8,191605,191499,5
2,4151,57018400,50695578,3693,2497,4,556,556,4,5,4,0,0,0,0,0,6,17621,17397,2
3,89,11979,6315683,267,86,2,278,278,2,5,1,0,0,0,0,0,5,823,718,5
4,93,13117,13243,96,101,4,556,556,4,5,3,0,0,0,0,0,6,577,485,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37406,13,1086,15995,122,126,1,70,348,3,5,1,0,0,0,0,0,4,237,179,0
37407,3550,69443446,101119394,5296,2132,0,0,560,4,5,2,0,0,0,0,0,4,18920,18696,2
37408,1046,17005995,71250,665,582,122,4333934,556,4,5,3,0,0,0,0,0,505,2685,2558,1
37409,486,62404,19004850,1474,631,4,556,3020765,84,5,1,0,0,0,0,0,6,5266,5143,1


In [16]:
Y = Y.astype('int')

In [17]:
print(X)

       Received Packets  Received Bytes  Sent Bytes  Sent Packets  \
0                   579         6350091    25264075          1157   
1                   480        12616018     6342453           578   
2                  4151        57018400    50695578          3693   
3                    89           11979     6315683           267   
4                    93           13117       13243            96   
...                 ...             ...         ...           ...   
37406                13            1086       15995           122   
37407              3550        69443446   101119394          5296   
37408              1046        17005995       71250           665   
37409               486           62404    19004850          1474   
37410             46222         2504813     8451973         39766   

       Port alive Duration (S)  Packets Rx Dropped  Packets Tx Dropped  \
0                          386                   0                   0   
1                      

In [18]:
print(Y)

0        1
1        5
2        2
3        5
4        0
        ..
37406    0
37407    2
37408    1
37409    1
37410    5
Name: Actual Label, Length: 37411, dtype: int64


In [19]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=.2,random_state =42)


In [20]:
X_train.shape,X_test.shape, Y_train.shape, Y_test.shape

((29928, 30), (7483, 30), (29928,), (7483,))

In [21]:
print(X_train)

       Received Packets  Received Bytes  Sent Bytes  Sent Packets  \
5472                130            9089     6322939           324   
17981              3241        31790293    12842671          2185   
22521               811         6353115    12618767           436   
5576              69195        16324861     3750635         69198   
35637              2794        50524303    82270946          5081   
...                 ...             ...         ...           ...   
16850               176           16587        9160            67   
6265                161           21400       21400           161   
11284            133582        19811475    45022461        134035   
860                 723        12632865    18982861          1202   
15795               616        12634664    12641144           705   

       Port alive Duration (S)  Packets Rx Dropped  Packets Tx Dropped  \
5472                       156                   0                   0   
17981                  

In [22]:
# training dataset ....
classifier = DecisionTreeClassifier()
classifier.fit(X_train,Y_train)


DecisionTreeClassifier()

In [23]:
prediction_on_training_data = classifier.predict(X_train)
accuracy_on_training_data = accuracy_score(Y_train,prediction_on_training_data)

In [24]:
print(accuracy_on_training_data)

1.0


In [25]:
Y_pred = classifier.predict(X_test)

In [26]:
accuracy_on_test_data = accuracy_score(Y_test,Y_pred)

In [27]:
print(accuracy_on_test_data)

1.0


In [28]:
print(Y_test)

32973    0
14163    3
21963    4
15633    2
31461    2
        ..
32725    2
31660    1
4716     2
6872     3
18832    0
Name: Actual Label, Length: 7483, dtype: int64


In [29]:
print(Y_pred)

[0 3 4 ... 2 3 0]


In [30]:
Y_test.unique()

array([0, 3, 4, 2, 1, 5])

In [ ]:
index = int(input())
print(Y_pred[index])
print(Y_test.iloc[[index]])


In [ ]:
plt.scatter(Y_test,Y_pred)
plt.title("Actual Y vs Predicted Y")
plt.xlabel('Actual y')
plt.ylabel('Predicted y')
plt.show()

In [ ]:
plot_confusion_matrix(classifier,X_test,Y_pred)
plt.show()

In [ ]:
index = int(input())
pred_value = Y_pred[index]
if pred_value == 0:
    print('attack type : '+'TCP-SYN Attack')
elif pred_value == 1:
    print('attack type :' +'Blackhole Attack')
elif pred_value == 2:
    print('attack type :' +'Diversion Attack')
elif pred_value == 3:
    print('attack type :' +'Overflow Attack')
elif pred_value == 5:
    print('attack type :' +'Portscan Attack')
else:
    print('attack type:' +' Normal')